In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import mibitrans as mbt
from mibitrans.visualize.animation import animate_1d

# Exact solution versus untruncated solution

The mibitrans solution uses the exact solution of the 3D transport ADE, as described in Wexler et al. (1992). The anatrans solution approximates this solution by the method described in Domenico (1987), but without truncating the x part of the equation. Both are adapted for multiple source zones and source depletion, see equations below. The approximation for transverse dispersivity terms in the anatrans solution introduces an error, of which the size depends on parameter choices.

## Anatrans solution
untruncated Domenico (1987)
\begin{aligned}
    C(x, y, z, t) &= \sum_{i=1}^{n}\left\{ \frac{C^*_{0,i}}{8} \exp \left[-k_s \left(t-\frac{xR}{v} \right)\right] \right. \\
    &\quad \cdot \left( \exp \left[ \frac{x\left(1-\sqrt{1+4\mu \alpha_x/v}\right)}{2\alpha_x}\right] \right. \\
    &\quad \quad \cdot \operatorname{erfc} \left[ \frac{x - vt\sqrt{1+4\mu \alpha_x/v}}{2\sqrt{\alpha_x vt }} \right] \\
    &\quad \ \, + \exp \left[ \frac{x\left(1+\sqrt{1+4\mu \alpha_x/v}\right)}{2\alpha_x}\right] \\
    &\quad \quad \cdot \left. \operatorname{erfc} \left[ \frac{x + vt\sqrt{1+4\mu \alpha_x/v}}{2\sqrt{\alpha_x vt }} \right] \right) \\
    &\quad \cdot \left( \operatorname{erf} \left[ \frac{y + Y_i/2}{2\sqrt{\alpha_y x}} \right] - \operatorname{erf} \left[ \frac{y - Y_i/2}{2\sqrt{\alpha_y x)}} \right] \right) \\
    &\quad \cdot \left. \left( \operatorname{erf} \left[ \frac{Z}{2\sqrt{\alpha_z x)}} \right] - \operatorname{erf} \left[ \frac{-Z}{2\sqrt{\alpha_z x}} \right] \right) \right\}
\end{aligned}


## Mibitrans solution
Wexler (1992)
$$
\begin{equation}\tag{2}
\begin{aligned}
    C(x,y,z,t) &= \sum_{i=1}^{n}\left(C^*_{0,i}\frac{x}{8\sqrt{\pi \alpha_{x}\frac{v\tau}{R}}}\exp(-k_s t) \right. \\
    &\quad \cdot \int_{0}^{t}\left[\frac{1}{\tau^{\frac{3}{2}}} \exp\left((k_s - \mu)\tau - \frac{(x-\frac{v\tau}{R})^2}{4\alpha_{x}\frac{v\tau}{R}}\right) \right. \\
    &\quad \cdot \left\{\operatorname{erfc}\left(\frac{y-Y_i}{2 \sqrt{\alpha_{y}\frac{v\tau}{R}}}\right)-\operatorname{erfc}\left(\frac{y+Y_i}{2 \sqrt{\alpha_{y}\frac{v\tau}{R}}}\right) \right\} \\
    &\quad \left. \left. \cdot \left\{\operatorname{erfc}\left(\frac{-Z}{2 \sqrt{\alpha_{z}\frac{v\tau}{R}}}\right)-\operatorname{erfc}\left(\frac{Z}{2 \sqrt{\alpha_{z}\frac{v\tau}{R}}}\right) \right\}\right] d\tau \right)
\end{aligned}
\end{equation}
$$

In [ ]:
hydro = mbt.HydrologicalParameters(
    velocity=0.277, # Flow velocity [m/d]
    porosity=0.25,  # Effective soil porosity [-]
    alpha_x=10,     # Longitudinal dispersivity, in [m]
    alpha_y=1,      # Transverse horizontal dispersivity, in [m]
    alpha_z=0.1     # Transverse vertical dispersivity, in [m]
)

att = mbt.AttenuationParameters(
    retardation=1,
    # Molecular diffusion, in [m2/day]
    diffusion=0,
    # Contaminant half life, in [days]
    half_life=0#5*365
)

source = mbt.SourceParameters(
    source_zone_boundary=np.array([10]),
    source_zone_concentration=np.array([11]),
    depth=2.5,
    total_mass="inf"
)

model = mbt.ModelParameters(
    # Model extent in the longitudinal (x) direction in [m].
    model_length = 800,
    # Model extent in the transverse horizontal (y) direction in [m].
    model_width = 150,
    # Model duration in [days].
    model_time = 10*365,
    # Model grid discretization step size in the longitudinal (x) direction, in [m].
    dx = 2,
    # Model grid discretization step size in the transverse horizontal (y) direction, in [m].
    dy = 1,
    # Model time discretization step size, in [days]
    dt = 25
)

In [ ]:
mbt_object = mbt.Mibitrans(hydro, att, source, model)
mbt_results = mbt_object.run()

ana_object = mbt.Anatrans(hydro, att, source, model)
ana_results = ana_object.run()

In [ ]:
%matplotlib ipympl

In [ ]:
ani = animate_1d(x_axis_parameter=mbt_results.y,
                 y_axis_parameter=[mbt_results.cxyt[:,:,25], mbt_results.cxyt[:,:,50],
                                   mbt_results.cxyt[:,:,100],mbt_results.cxyt[:,:,200],
                                   ana_results.cxyt[:,:,25], ana_results.cxyt[:,:,50],
                                   ana_results.cxyt[:,:,100], ana_results.cxyt[:,:,200]],
                 time_parameter=mbt_results.t,
                 y_colors=["darkgreen", "limegreen", "cornflowerblue", "blue",
                           "green", "greenyellow", "darkturquoise", "dodgerblue"],
                 y_names=["Mibitrans x=50m", "Mibitrans x=100m", "Mibitrans x=200m", "Mibitrans x=400m",
                          "Anatrans x=50m", "Anatrans x=100m", "Anatrans x=200m", "Anatrans x=400m"],
                 linestyle=["-", "-", "-", "-", "--", "--", "--", "--"])
plt.xlabel("y-position [m]")
plt.ylabel(r"Concentration [$g/m^3$]")
#ani.save("transverse_animation_2.gif", fps=10)
plt.show()


In [ ]:
ani1 = animate_1d(x_axis_parameter=mbt_results.x,
                  y_axis_parameter=[mbt_results.cxyt[:,75,:], mbt_results.cxyt[:,84,:],
                                    mbt_results.cxyt[:,100,:], mbt_results.cxyt[:,125,:],
                                   ana_results.cxyt[:,75,:], ana_results.cxyt[:,84,:],
                                    ana_results.cxyt[:,100,:], ana_results.cxyt[:,125,:]],
                  time_parameter=mbt_results.t,
                  y_colors=["darkgreen", "limegreen", "cornflowerblue", "blue",
                            "green", "greenyellow", "darkturquoise", "dodgerblue"],
                  y_names=["Mibitrans y=0m", "Mibitrans y=9m", "Mibitrans y=25m", "Mibitrans y=50m",
                           "Anatrans y=0m", "Anatrans y=9m", "Anatrans y=25m", "Anatrans y=50m"],
                  linestyle=["-", "-", "-", "-", "--", "--", "--", "--"])
plt.xlabel("x-position [m]")
plt.ylabel(r"Concentration [$g/m^3$]")
plt.show()


In [ ]:
plt.figure()
mbt_results.breakthrough(50, 0, color="darkgreen", linestyle="-", label="Mibitrans 50m")
mbt_results.breakthrough(100, 0, color="limegreen", linestyle="-", label="Mibitrans 100m")
mbt_results.breakthrough(200, 0, color="cornflowerblue", linestyle="-", label="Mibitrans 200m")
mbt_results.breakthrough(400, 0, color="blue", linestyle="-", label="Mibitrans 400m")
ana_results.breakthrough(50, 0, color="green", linestyle="--", label="Anatrans 50m")
ana_results.breakthrough(100, 0, color="greenyellow", linestyle="--", label="Anatrans 100m")
ana_results.breakthrough(200, 0, color="darkturquoise", linestyle="--", label="Anatrans 200m")
ana_results.breakthrough(400, 0, color="dodgerblue", linestyle="--", label="Anatrans 400m")
plt.title("Breakthrough plot of Mibitrans and Anatrans model, at various x locations.")
plt.legend()
plt.show()

#### Comparing model differences for various flow velocities and dispersivities

In [ ]:
parameter_list = [0.1, 0.5, 1, 5, 10]
output_list_mbt = []
output_list_ana = []
for par in parameter_list:
    print(f"starting run with par = {par}")
    mbt_object_alpha = mbt.Mibitrans(hydro, att, source, model)
    mbt_object_alpha.hydrological_parameters.alpha_x = par
    mbt_object_alpha.hydrological_parameters.alpha_y = par / 10
    mbt_object_alpha.hydrological_parameters.alpha_z = par / 100
    mbt_res_alp = mbt_object_alpha.run()
    output_list_mbt.append(mbt_res_alp.cxyt)

    ana_object_alpha = mbt.Anatrans(hydro, att, source, model)
    ana_object_alpha.hydrological_parameters.alpha_x = par
    ana_object_alpha.hydrological_parameters.alpha_y = par / 10
    ana_object_alpha.hydrological_parameters.alpha_z = par / 100
    ana_res_alp = ana_object_alpha.run()
    output_list_ana.append(ana_res_alp.cxyt)

In [ ]:
colors_mbt = ["darkgreen", "limegreen", "cornflowerblue", "blue", "indigo"]
colors_ana = ["green", "greenyellow", "darkturquoise", "dodgerblue", "darkviolet"]
plt.figure()
for i, conc in enumerate(output_list_mbt):
    plt.plot(mbt_res_alp.x, conc[-1,75,:], color=colors_mbt[i],
             label=f"Mibitrans a={parameter_list[i]}m")
for i, conc in enumerate(output_list_ana):
    plt.plot(ana_res_alp.x, conc[-1,75,:], color=colors_ana[i], linestyle="--",
             label=f"Anatrans a={parameter_list[i]}m")
plt.legend()
plt.show()

In [ ]:
colors_mbt = ["darkgreen", "limegreen", "cornflowerblue", "blue", "indigo"]
colors_ana = ["green", "greenyellow", "darkturquoise", "dodgerblue", "darkviolet"]
plt.figure()
for i, conc in enumerate(output_list_mbt):
    plt.plot(mbt_res_alp.y, conc[-1,:,175], color=colors_mbt[i],
             label=f"Mibitrans a={parameter_list[i]}m")
for i, conc in enumerate(output_list_ana):
    plt.plot(ana_res_alp.y, conc[-1,:,175], color=colors_ana[i],
             linestyle="--", label=f"Anatrans a={parameter_list[i]}m")
plt.legend()
plt.show()

In [ ]:
parameter_list_v = [0.1, 0.1, 0.1, 0.1, 0.1]
parameter_list_a = [0., 5, 2, 1, 0.5]
output_list_mbt_v = []
output_list_ana_v = []
for i in range(len(parameter_list_v)):
    parv = parameter_list_v[i]
    para = parameter_list_a[i]
    print(f"starting run with parv = {parv} and para = {para}")
    mbt_va = mbt.Mibitrans(hydro, att, source, model)
    mbt_va.hydrological_parameters.velocity = parv
    mbt_va.hydrological_parameters.alpha_x = para
    mbt_va.hydrological_parameters.alpha_y = para / 10
    mbt_va.hydrological_parameters.alpha_z = 0#para / 100
    mbt_res_va = mbt_va.run()
    output_list_mbt_v.append(mbt_res_va.relative_cxyt)

    ana_va = mbt.Anatrans(hydro, att, source, model)
    ana_va.hydrological_parameters.velocity = parv
    ana_va.hydrological_parameters.alpha_x = para
    ana_va.hydrological_parameters.alpha_y = para / 10
    ana_va.hydrological_parameters.alpha_z = 0#para / 100
    ana_res_va = ana_va.run()
    output_list_ana_v.append(ana_res_va.relative_cxyt)

In [ ]:
colors_mbt = ["darkgreen", "limegreen", "cornflowerblue", "blue", "indigo"]
colors_ana = ["green", "greenyellow", "darkturquoise", "dodgerblue", "darkviolet"]
plt.figure()
for i, conc in enumerate(output_list_mbt_v):
    plt.plot(mbt_res_va.x, conc[-1,75,:], color=colors_mbt[i],
             label=f"Mbt v={parameter_list_v[i]}m/d, a={parameter_list_a[i]}m")
for i, conc in enumerate(output_list_ana_v):
    plt.plot(ana_res_va.x, conc[-1,75,:], color=colors_ana[i],
             linestyle="--", label=f"Ana v={parameter_list_v[i]}m/d, a={parameter_list_a[i]}m")
plt.xlim((0,300))
plt.legend()
plt.show()

In [ ]:
colors_mbt = ["darkgreen", "limegreen", "cornflowerblue", "blue", "indigo"]
colors_ana = ["green", "greenyellow", "darkturquoise", "dodgerblue", "darkviolet"]
plt.figure()
for i, conc in enumerate(output_list_mbt_v):
    plt.plot(mbt_res_va.y, conc[-1, :, 75], color=colors_mbt[i], label=f"Mibitrans a={parameter_list_v[i]}m")
for i, conc in enumerate(output_list_ana_v):
    plt.plot(ana_res_va.y, conc[-1, :, 75], color=colors_ana[i], linestyle="--",
             label=f"Anatrans a={parameter_list[i]}m")
plt.legend()
plt.show()

In [ ]:
mbtv=output_list_mbt_v
anav=output_list_ana_v
ani = animate_1d(x_axis_parameter=mbt_res_va.x,
                 y_axis_parameter=[mbtv[0][:,75,:], mbtv[1][:,75,:],mbtv[2][:,75,:],
                                   mbtv[3][:,75,:], mbtv[4][:,75,:],
                                   anav[0][:,75,:], anav[1][:,75,:], anav[2][:,75,:],
                                   anav[3][:,75,:], anav[4][:,75,:]],
                 time_parameter=mbt_res_va.t,
                 y_colors=["darkgreen", "limegreen", "cornflowerblue", "blue", "indigo",
                           "green", "greenyellow", "darkturquoise", "dodgerblue", "violet"],
                 y_names=[r"Mibitrans $\alpha_l=10$", r"Mibitrans $\alpha_l=5$", r"Mibitrans $\alpha_l=1$",
                          r"Mibitrans $\alpha_l=0.5$", r"Mibitrans $\alpha_l=0.1$", r"Anatrans $\alpha_l=10$",
                          r"Anatrans $\alpha_l=5$", r"Anatrans $\alpha_l=1$", r"Anatrans $\alpha_l=0.5$",
                          r"Anatrans $\alpha_l=0.1$",
                        ],
                 linestyle=["-", "-", "-", "-", "-", "--", "--", "--", "--", "--"])
plt.xlabel("x-position [m]")
plt.ylabel(r"Concentration [$g/m^3$]")
plt.xlim((0,350))
plt.show()
#ani.save("centerline_animation.gif", fps=10)

In [ ]:
mbtv=output_list_mbt_v
anav=output_list_ana_v
ani = animate_1d(x_axis_parameter=mbt_results.y,
                 y_axis_parameter=[mbtv[0][:,:,50], mbtv[1][:,:,50], mbtv[2][:,:,50],
                                   mbtv[3][:,:,50], mbtv[4][:,:,50],
                                   anav[0][:,:,50], anav[1][:,:,50], anav[2][:,:,50],
                                   anav[3][:,:,50], anav[4][:,:,50]],
                 time_parameter=mbt_res_va.t,
                 y_colors=["darkgreen", "limegreen", "cornflowerblue", "blue", "indigo",
                           "green", "greenyellow", "darkturquoise", "dodgerblue", "violet"],
                 y_names=[r"Mibitrans $\alpha_l=10$", r"Mibitrans $\alpha_l=5$", r"Mibitrans $\alpha_l=1$",
                          r"Mibitrans $\alpha_l=0.5$", r"Mibitrans $\alpha_l=0.1$", r"Anatrans $\alpha_l=10$",
                          r"Anatrans $\alpha_l=5$", r"Anatrans $\alpha_l=1$", r"Anatrans $\alpha_l=0.5$",
                          r"Anatrans $\alpha_l=0.1$",
                        ],
                 linestyle=["-", "-", "-", "-", "-", "--", "--", "--", "--", "--"])
plt.xlabel("y-position [m]")
plt.ylabel(r"Concentration [$g/m^3$]")
#ani.save("transverse_animation.gif", fps=10)
plt.show()

In [ ]:
from matplotlib import animation

In [ ]:
plt.clf()
mbt_a10 = mbtv[0]
ana_a10 = anav[0]

err_aprx_raw = (ana_a10 - mbt_a10) / mbt_a10 * 100
err_aprx_raw = np.where(np.isnan(err_aprx_raw), 0, err_aprx_raw)
err_aprx = np.log10(abs(err_aprx_raw))
err_abs = ana_a10 - mbt_a10
# print(err_aprx[-1,len(mbt_va.y)//2, :])
# plt.plot(mbt_va.x, mbt_a10[-1,len(mbt_va.y)//2, :], color = "green")
# plt.plot(mbt_va.x, ana_a10[-1,len(mbt_va.y)//2, :], color = "red")
# plt.show()
#
# plt.plot(mbt_va.x, err_aprx[-1,len(mbt_va.y)//2, :], color = "blue")
# plt.show()
# plt.pcolormesh(mbt_va.x, mbt_va.y, err_abs[10, :, :], cmap="jet")
# plt.colorbar()
# plt.show()

fig, ax = plt.subplots()
mesh = ax.pcolormesh(
    mbt_va.x[:150], mbt_va.y[35:115], err_abs[0, 35:115, :150], vmin=-0.2, vmax=0.2, cmap="Spectral"
)
cbar = fig.colorbar(mesh, ax=ax)
cbar.set_label("Absolute error in rel. concentration")
ax.set_xlabel("Distance from source (m)")
ax.set_ylabel("Distance from plume center (m)")

def update(frame):
    """Update animation frame to the next frame."""
    mesh.set_array(err_abs[frame, 35:115, :150])
    ax.set_title(f"Concentration distribution at t={mbt_va.t[frame]} days")
    return mesh

ani = animation.FuncAnimation(fig=fig, func=update, frames=len(mbt_va.t))
plt.show()
